In [1]:
import pandas as pd

Считываем данные

In [2]:
data = pd.read_csv('archive-3/rating.csv')

In [684]:
data

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40
...,...,...,...,...
20000258,138493,68954,4.5,2009-11-13 15:42:00
20000259,138493,69526,4.5,2009-12-03 18:31:48
20000260,138493,69644,3.0,2009-12-07 18:10:57
20000261,138493,70286,5.0,2009-11-13 15:42:24


In [178]:
movie = pd.read_csv('archive-3/movie.csv')
movie.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [4]:
links = pd.read_csv('archive-3/link.csv')

print(len(links))
print(len(links.movieId.unique()))

27278
27278


In [179]:
movie = pd.concat([movie, links.drop(['movieId'], axis=1)], axis=1)

In [180]:
data = data[data.movieId.isin(movie.movieId.unique())]

In [181]:
movie = movie[movie.movieId.isin(data.movieId.unique())]

In [182]:
print(len(movie.movieId.unique()))
print(len(data.movieId.unique()))

26744
26744


In [26]:
genome = pd.read_csv('archive-3/genome_scores.csv')

In [217]:
genome = genome[genome.movieId.isin(movie.movieId.unique())]
grouped = genome.groupby('movieId').agg({'relevance': ['max']}).reset_index()

In [161]:
tags = pd.Series([0]*len(grouped))
for i in range(len(grouped.values)):
    if len(genome[(genome.movieId == grouped.values[i][0]) & (genome.relevance == grouped.values[i][1])].tagId) > 1:
        tags[i] = genome[(genome.movieId == grouped.values[i][0]) & (genome.relevance == grouped.values[i][1])].tagId.reset_index().loc[0, 'tagId']
    else:
        tags[i] = genome[(genome.movieId == grouped.values[i][0]) & (genome.relevance == grouped.values[i][1])].tagId

In [189]:
movie = movie.reset_index()

In [190]:
movie['tagId'] = [0] * len(movie)
indexes = np.array([i for i in range(len(movie))])
indexes = indexes[movie.movieId.isin(grouped.movieId.unique())]
for i in range(len(indexes)):
    movie.loc[indexes[i], 'tagId'] = tags[i]
movie

,index,movieId,title,genres,imdbId,tmdbId,tagId
0,0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,1036
1,1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,8844.0,29
2,2,3,Grumpier Old Men (1995),Comedy|Romance,113228,15602.0,451
3,3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885,31357.0,1116
4,4,5,Father of the Bride Part II (1995),Comedy,113041,11862.0,451
...,...,...,...,...,...,...,...
26739,27273,131254,Kein Bund für's Leben (2007),Comedy,466713,4436.0,0
26740,27274,131256,"Feuer, Eis & Dosenbier (2002)",Comedy,277703,9274.0,0
26741,27275,131258,The Pirates (2014),Adventure,3485166,285213.0,0
26742,27276,131260,Rentun Ruusu (2001),(no genres listed),249110,32099.0,0


Разобьем данные на test и train части, в train войдут более ранние записи, в test более поздние, это моделирует реальную жизнь: мы делаем предсказания на основе более поздних записей

In [212]:
test = pd.DataFrame(columns=data.columns)
train = pd.DataFrame(columns=data.columns)
test_ratio = 0.3

temp = data.sort_values('timestamp').reset_index()
temp.drop('index', axis=1, inplace=True)

dummy_test = temp.iloc[int(len(data) * (1.0 - test_ratio)):]
dummy_train = temp.iloc[: int(len(data) * (1.0 - test_ratio))]

print(len(dummy_test))
print(len(dummy_train))

test = pd.concat([test, dummy_test])
train = pd.concat([train, dummy_train])

6000079
14000184


In [10]:
!pip install surprise

In [11]:
from surprise.prediction_algorithms import SVD
from surprise import Reader
from surprise import Dataset

In [12]:
reader = Reader(rating_scale = (0.5, 5))

train_svd = Dataset.load_from_df(train.drop(['timestamp'], axis=1), reader)
test_svd = Dataset.load_from_df(test.drop(['timestamp'], axis=1), reader)

In [13]:
import numpy as np

In [14]:
train_svd = train_svd.build_full_trainset()

In [17]:
testset = test_svd.df.values.tolist()

В качестве метрик я выбрала mse и r2, поскольку наши модели построены так, что они пытаются предсказать оценку конкретного пользователя конкретного товара, а не пытаются получить топ n рекомендаций для пользователя

In [20]:
from sklearn.metrics import r2_score, mean_squared_error

In [216]:
epoch_list = [5, 10]
lr_list = [0.001, 0.003, 0.005]
best_r2_score = -10000000000
best_mse = 10000000000
for epoch in epoch_list:
    for lr in lr_list:
        svd = SVD(n_factors=50, n_epochs=epoch, lr_all=lr)
        svd.fit(train_svd)
        print('fitted')
        pred = svd.test(testset)
        print('predicted')
        ypred = []
        for i in range(len(pred)):
            ypred.append(pred[i].est)
        r2 = r2_score(test.rating.to_list(), ypred)
        mse = mean_squared_error(ypred, test.rating.to_list())
        print(r2, mse)
        if r2 > best_r2_score:
            best_r2_score = r2
        if mse < best_mse:
            best_mse = mse

print('mse: ', best_mse)
print('r2_score: ', best_r2_score)

fitted
predicted
0.150539686641882 0.8776921927599622
fitted
predicted
0.15491810557494579 0.8731682567340513
fitted
predicted
0.15780694729932576 0.8701834041308919
fitted
predicted
0.15083573452401178 0.8773863057035073
fitted
predicted
0.15795655455883528 0.8700288246625084
fitted
predicted
0.15987646902608987 0.8680451016890187
mse:  0.8680451016890187
r2_score:  0.15987646902608987


In [191]:
movie.isna().sum().sum()

243

In [192]:
movie.tmdbId.isna().sum()

243

In [193]:
movie = movie.fillna(movie.tmdbId.mean())

In [194]:
movie.isna().sum().sum()

0

In [195]:
for title in movie.title:
    if title[-1] != ")":
        print(title)
        print(title[-1])

11'09\\"01 - September 11 (2002)"
"
Babe Ruth Story, The (1948) 
 
Heroes of Telemark, The (1965) 
 
Babylon 5
5
\\"Great Performances\"\" Cats (1998)"
"
Truth, The (2010) 
 
Wedding Song, The (2008) 
 
White Hell of Pitz Palu, The (Die weiße Hölle vom Piz Palü) (1929) 
 
Pool, The (Swimming Pool - Der Tod feiert mit) (2001) 
 
Possession of David O'Reilly, The (2010) 
 
Horrible Way to Die, A (2010) 
 
Legend of Suram Fortress, The (Ambavi Suramis tsikhitsa) (1986) 
 
Silence Before Bach, The (Die Stille vor Bach) (2007) 
 
Poseidon Adventure, The (2005) 
 
Nun, The (La monja) (2005) 
 
Eleventh Year, The (Odinnadtsatyy) (1928) 
 
Fairy, The (La fée) (2011) 
 
Lost, Lost, Lost (1976) 
 
Craigslist Killer, The (2011) 
 
Rites of May, The (Itim) (1977) 
 
Age of the Medici, The (L'età di Cosimo de Medici) (1973) 
 
Den tatuerade änkan (Tattooed Widow, The) (1998) 
 
Woman's Face, A (En kvinnas ansikte) (1938) 
 
Act of Seeing with One's Own Eyes, The (1971) 
 
Net 2.0, The (2006) 
 
Wor

In [196]:
def f(x):
    i = -1
    while x[i] == " ":
        i -= 1
    if x[i] != ')':
        i -= 1
    if x[i] == ')' and x[i - 1].isdigit():
        return int(x[-4 + i:i])
    return np.NaN

In [197]:
movie['year'] = movie.title.apply(lambda x: f(x))

In [198]:
movie['title'] = movie.title.apply(lambda x: x.split('(')[0])

In [199]:
movie = movie.fillna(movie.year.mean())

In [71]:
from lightfm.data import Dataset

dataset = Dataset()
dataset.fit(data.userId.unique(), 
         data.movieId.unique()
        )
train_matrix, weights_matrix = dataset.build_interactions([tuple(i) for i in train.drop(['timestamp'], axis=1).values])
test_matrix, weights_matrix = dataset.build_interactions([tuple(i) for i in test.drop(['timestamp'], axis=1).values])

/Users/macbook/opt/anaconda3/lib/python3.8/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [200]:
train_features = movie[movie.movieId.isin(train.movieId.unique())]

In [201]:
def feature_colon_value(my_list):
    result = []
    ll = ['tmdbId', 'title', 'year', 'imdbId', 'tagId']
    aa = my_list
    for x,y in zip(ll,aa):
        res = str(x) +":"+ str(y)
        result.append(res)
    return result

In [202]:
ad_subset = train_features[['tmdbId', 'title', 'year', 'imdbId', 'tagId']]
ad_list = [list(x) for x in ad_subset.values]
feature_list = []
for item in ad_list:
    feature_list.append(feature_colon_value(item))    

In [203]:
item_tuple = list(zip(train_features.movieId, feature_list))

In [204]:
uf = []
col = ['imdbId']*len(movie.imdbId.unique()) + ['tmdbId']*len(movie.tmdbId.unique()) + ['year']*len(movie.year.unique()) + ['title']*len(movie.title.unique())+ ['tagId']*len(movie.tagId.unique())
unique_f1 = list(movie.imdbId.unique()) + list(movie.tmdbId.unique()) + list(movie.year.unique()) + list(movie.title.unique()) + list(movie.tagId.unique())

for x,y in zip(col, unique_f1):
    res = str(x)+ ":" +str(y)
    uf.append(res)

In [205]:
dataset.fit_partial(item_features=uf)

In [206]:
train_features = dataset.build_item_features(item_tuple, normalize= False)

In [88]:
train_features.toarray()

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [207]:
test_features = movie[movie.movieId.isin(test.movieId.unique())]

In [208]:
ad_subset = test_features[['tmdbId', 'title', 'year', 'imdbId', 'tagId']]
ad_list = [list(x) for x in ad_subset.values]
feature_list = []
for item in ad_list:
    feature_list.append(feature_colon_value(item))  
item_tuple = list(zip(test_features.movieId, feature_list))

In [209]:
test_features = dataset.build_item_features(item_tuple, normalize= False)

In [94]:
print(train_matrix.toarray().shape)
print(test_matrix.toarray().shape)

(138493, 26744)
(138493, 26744)


In [208]:
!pip3 install lightfm

     |████████████████████████████████| 310 kB 809 kB/s eta 0:00:01
  Created wheel for lightfm: filename=lightfm-1.16-cp38-cp38-macosx_10_9_x86_64.whl size=446297 sha256=3570adfa6fba2d532c46964f40ddf68a7594e15c56bd28287efcba3b3c0a4387
  Stored in directory: /Users/macbook/Library/Caches/pip/wheels/ec/bb/51/9c487d021c1373b691d13cadca0b65b6852627b1f3f43550fa
Successfully built lightfm


In [95]:
from lightfm import LightFM

In [96]:
from lightfm.evaluation import auc_score

In [97]:
test = test[(test.movieId <= 26665) & (test.userId <= 138487)]

In [210]:
loss_list = ['logistic', 'bpr', 'warp', 'warp-kos']
lr_list = [0.001, 0.003, 0.005]
component_list = [25, 50, 75]
best_mse = 1000000000
best_r2 = -1000000000
for loss in loss_list:
    for lr in lr_list:
        for component in component_list:
            lfm = LightFM(no_components=component, loss=loss, learning_rate=lr)
            lfm.fit(train_matrix, item_features=train_features)
            pred = lfm.predict(np.array(test.userId.tolist()), np.array(test.movieId.tolist()))
            r2 = r2_score(test.rating.to_list(), pred)
            mse = mean_squared_error(test.rating.to_list(), pred)
            if mse < best_mse:
                best_mse = mse
            if r2 > best_r2:
                best_r2 = r2
            print(mse, r2)
print('best_mse: ', best_mse)
print('best_r2: ', best_r2)

13.293961523607713 -11.577102262187385
13.29395748367812 -11.577098440105642
13.293972669258258 -11.577112806823491
12.866994868391435 -11.173159218147196
12.86713582162033 -11.173292570659978
12.867114585033312 -11.173272479227776
12.666989420809019 -10.98393879932986
12.66764460946053 -10.984558657801424
12.667251872593996 -10.984187098750064
13.865258135656617 -12.117592461366954
13.86516218172968 -12.117501681628392
13.865576610487961 -12.117893762864977
14.052808216722168 -12.295029152804197
14.052123477300709 -12.294381337048799
14.051005125093889 -12.293323290502977
14.225605200305488 -12.458508287993924
14.219304372789333 -12.452547224253342
14.214349804818532 -12.447859824795847
13.687350673015496 -11.94927842293152
13.664373892337512 -11.927540634708674
13.639220327254165 -11.90374344229541
13.797165893377574 -12.053172003034472
13.773946772628546 -12.031204942607825
13.761780149955511 -12.019694388942355
13.964213704643585 -12.211211982406779
13.94973118041968 -12.1975104091

Оценивая метрики, которые мы получили на двух моделей, можно сделать вывод, что SVD алгоритм лучше справляется с тем, чтобы делать предсказания